In [1]:
import math
def make_verbose(start_with, done, total, max_lines=100):
    num_lines = int((done/total)*max_lines)
    lines = "="*num_lines
    spaces = " "*(max_lines-num_lines)
    print(f"\r{start_with}  {lines}>{spaces}  {done}/{total}", end="", sep=" ", flush=True)

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import os
import time

In [3]:
url="https://supremo.nic.in/KnowYourOfficerIAS.aspx"

driver = webdriver.Chrome('C:/Users/vedan/Desktop/python-utils/chromedriver/chromedriver.exe')
# driver_profile = webdriver.Chrome('C:/Users/vedan/Desktop/python-utils/chromedriver/chromedriver.exe')

In [4]:
driver.get(url)

In [5]:
DROPDOWN_XPATH = "/html/body/form/section[2]/div/div[1]/section/div/div/div[2]/div/div[1]/div/div/div[2]/div[1]/div/div/ul"

OPTIONS_XPATH = "/html/body/form/section[2]/div/div[1]/section/div/div/div[2]/div/div[1]/div/div/div[2]/div[1]/div/div/div/ul"

YEAR_INPUT = "/html/body/form/section[2]/div/div[1]/section/div/div/div[2]/div/div[1]/div/div/div[1]/div/div/input[1]"

SUBMIT_BUTTON = "/html/body/form/section[2]/div/div[1]/section/div/div/div[2]/div/div[2]/input[1]"

RESET_BUTTON = "/html/body/form/section[2]/div/div[1]/section/div/div/div[2]/div/div[2]/input[2]"

TABLE = "/html/body/form/section[2]/div/div[1]/section/div/div/div[2]/div/div[4]/div[2]"

TABLE_PRIMARY = "/html/body/form/div[3]/div[2]/div/div/div/div[1]/table"

TABLE_SECONDARY = "/html/body/form/div[3]/div[2]/div/div/div/div[2]"

DATE_OF_JOINING_TABLE_ROW = "/html/body/form/section[2]/div/div[1]/section/div/div/div[2]/div/div[4]/div[2]/table/tbody/tr"
DATE_OF_JOINING_TABLE_ELEMENT = "/td[6]"

In [6]:
def set_search_query(year):
    driver.find_element_by_xpath(RESET_BUTTON).click()
    driver.find_element_by_xpath(DROPDOWN_XPATH).click()
    length = len(driver.find_element_by_xpath(OPTIONS_XPATH).find_elements_by_tag_name("li"))
    driver.find_element_by_xpath(OPTIONS_XPATH).find_elements_by_tag_name("li")[1].click()
    for i in range(2,length-1):
        driver.find_element_by_xpath(DROPDOWN_XPATH).click()
        driver.find_element_by_xpath(OPTIONS_XPATH).find_elements_by_tag_name("li")[i].click()
    
    year_input = driver.find_element_by_xpath(YEAR_INPUT)
    year_input.click()
    year_input.send_keys(str(year))

    driver.find_element_by_xpath(SUBMIT_BUTTON).click()

In [8]:
set_search_query(2018)

In [9]:
def extract_info_from_profile(url):
    info = {
        "Name":"",
        "Service":"",
        "Cadre":"",
        "Allotment Year":"",
        "Date of Birth":"",
        "Identity No.":"",
        "Source of Recruitment":"",
        "Gender":"",
        "Place of Domicile": "",
        "Mother Tongue":"",
        "Languages Known":"",
        "Retirement Reason":"",
        "Qualification/University/Institute":"",
        "Subject":"",
        "Division":"",
        "Designation/Level":"",
        "Ministry/Department/Office/Location":"",
        "Organisation":"",
        "Experience(major/minor)":"",
        "Period(From/To)":""
    }
    driver_profile.get(url)
    table_primary = driver_profile.find_element_by_xpath(TABLE_PRIMARY).find_elements_by_tag_name("tr")
    tables_secondary = driver_profile.find_element_by_xpath(TABLE_SECONDARY).find_elements_by_tag_name("table")

    t_identification = {
        "III. Educational Qualifications.": ["Qualification/University/Institute", "Subject", "Division"],
        "IV. Experience Details": ["Designation/Level", "Ministry/Department/Office/Location", "Organisation", "Experience(major/minor)", "Period(From/To)"]
    }

    for row in table_primary:
        tds = row.find_elements_by_tag_name("td")
        labels = tds[0].text.replace(":","").strip().split("/ ")
        values = tds[1].text.strip().split("/ ")
        for label, value in zip(labels, values):
            info[label] += value

    for table in tables_secondary[1:3]:
        thead = table.find_element_by_tag_name("thead")
        table_name = thead.find_element_by_tag_name("tr").text
        keys_to_address = t_identification.get(table_name, 0)
        try:
            trs = table.find_element_by_tag_name("tbody").find_elements_by_tag_name("tr")
            for j, row in enumerate(trs):
                if j%2 == 0:
                    end_string = ""
                    if j < (len(trs)-2):
                        end_string += "|"
                    items = row.find_elements_by_tag_name("td")[1:]
                    for i, key in enumerate(keys_to_address):
                        information_to_add = items[i].text.replace("\n", "/")
                        info[key] += information_to_add + end_string
        except:
            trs = None
            for i, key in enumerate(keys_to_address):
                information_to_add = ""
                info[key] += information_to_add
    return info

In [9]:
profile_url = "https://supremo.nic.in/ERSheetHtml.aspx?OffIDErhtml=27177&PageId="
extract_info_from_profile(profile_url)

{'Name': 'Shri J.D. Desai',
 'Service': 'Indian Administrative Service ',
 'Cadre': 'Gujarat ',
 'Allotment Year': '2009',
 'Date of Birth': '09/07/1963',
 'Identity No.': '01GJ815306',
 'Source of Recruitment': 'By Promotion from State Civil Service',
 'Gender': 'Male',
 'Place of Domicile': 'N.A.',
 'Mother Tongue': 'N.A.',
 'Languages Known': '',
 'Retirement Reason': 'Serving',
 'Qualification/University/Institute': '',
 'Subject': '',
 'Division': '',
 'Designation/Level': 'N.A./Not Available',
 'Ministry/Department/Office/Location': '',
 'Organisation': 'Cadre (AIS)',
 'Experience(major/minor)': '/ N.Applicable/N.Available',
 'Period(From/To)': '01/04/2016 -'}

In [10]:
def scrape_table(table, info, year):
    keys = [key for key in info.keys()]
    trs = table.find_elements_by_tag_name("tr")[1:]
    length = len(trs)

    for k, tr in enumerate(trs):
        make_verbose(year, k+1, length, 25)
        # td = tr.find_elements_by_tag_name("td")[0]
        # prof_url = tr.find_element_by_xpath(f"/html/body/form/section[2]/div/div[1]/section/div/div/div[2]/div/div[4]/div[2]/table/tbody/tr[{str(k+2)}]/td[1]/u/a").get_property("href")
        prof_url = tr.find_element_by_tag_name("a").get_property("href")
        info.get("Profile URL", 0).append(prof_url)
        # info.get("Date of Joining", 0).append(tr.find_element_by_xpath(f"/html/body/form/section[2]/div/div[1]/section/div/div/div[2]/div/div[4]/div[2]/table/tbody/tr[{str(k+2)}]/td[6]").text)
        # information = extract_info_from_profile(prof_url)

        # for key in information.keys():
        #     info.get(key, 0).append(information[key])


In [11]:
def scrape_for_range(start_year=1951, end_year=2020, info={}):
    for i in range(start_year, end_year+1):
        set_search_query(i)
        table = driver.find_element_by_xpath(TABLE)
        scrape_table(table, info, i)

In [16]:
driver.get(url)
start_year = 2020
end_year = 2020
info = {
    "Name":[],
    "Service":[],
    "Cadre":[],
    "Allotment Year":[],
    "Date of Birth":[],
    "Date of Joining":[],
    "Identity No.":[],
    "Source of Recruitment":[],
    "Gender":[],
    "Place of Domicile": [],
    "Mother Tongue":[],
    "Languages Known":[],
    "Retirement Reason":[],
    "Qualification/University/Institute":[],
    "Subject":[],
    "Division":[],
    "Designation/Level":[],
    "Ministry/Department/Office/Location":[],
    "Organisation":[],
    "Experience(major/minor)":[],
    "Period(From/To)":[],
    "Profile URL": []
}
scrape_for_range(start_year, end_year, info)
# info["Name"] = info["Name"][:len(info["Name"])-1]

2020  =========================>  122/122

In [13]:
for key in info.keys():
    print(f"{key}:{len(info[key])}")

Name:0
Service:0
Cadre:0
Allotment Year:0
Date of Birth:0
Date of Joining:0
Identity No.:0
Source of Recruitment:0
Gender:0
Place of Domicile:0
Mother Tongue:0
Languages Known:0
Retirement Reason:0
Qualification/University/Institute:0
Subject:0
Division:0
Designation/Level:0
Ministry/Department/Office/Location:0
Organisation:0
Experience(major/minor):0
Period(From/To):0
Profile URL:9570


In [14]:
def create_df_from_dict(info):
    df = pd.DataFrame()
    keys = [str(key) for key in info.keys()]
    for key in keys[:len(keys)-1]:
        print(key)
        df[key] = list(info[key])
    
    return df

In [15]:
profiles = info["Profile URL"]

In [17]:
profiles.append(info["Profile URL"])

In [86]:
new_df = create_df_from_dict(info)

Name
Service
Cadre
Allotment Year
Date of Birth
Date of Joining
Identity No.
Source of Recruitment
Gender
Place of Domicile
Mother Tongue
Languages Known
Retirement Reason
Qualification/University/Institute
Subject
Division
Designation/Level
Ministry/Department/Office/Location
Organisation
Experience(major/minor)
Period(From/To)


In [87]:
new_df

,Name,Service,Cadre,Allotment Year,Date of Birth,Date of Joining,Identity No.,Source of Recruitment,Gender,Place of Domicile,...,Languages Known,Retirement Reason,Qualification/University/Institute,Subject,Division,Designation/Level,Ministry/Department/Office/Location,Organisation,Experience(major/minor),Period(From/To)
0,Shri Jatin Kishore,Indian Administrative Service,A G M U T,2020,22/07/1994,,01UT119A01,Direct Recruitment,Male,Delhi,...,,Serving,,,,,,,,
1,Ms. Vishakha Yadav,Indian Administrative Service,A G M U T,2020,21/02/1993,,01UT119A02,Direct Recruitment,Female,Delhi,...,,Serving,,,,,,,,
2,Ms. Navneet Mann,Indian Administrative Service,A G M U T,2020,06/06/1995,,01UT119A03,Direct Recruitment,Female,Punjab,...,,Serving,,,,,,,,
3,Shri Shishir Gupta,Indian Administrative Service,A G M U T,2020,07/12/1990,,01UT119A04,Direct Recruitment,Male,Rajasthan,...,,Serving,,,,,,,,
4,Ms. Yashni Nagarajan,Indian Administrative Service,A G M U T,2020,23/03/1993,,01UT119A05,Direct Recruitment,Female,Arunachal Pradesh,...,,Serving,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,Ms. Gunjan Singh,Indian Administrative Service,Rajasthan,2020,29/07/1993,,01RJ119S02,Direct Recruitment,Female,Uttar Pradesh,...,,Serving,,,,,,,,
118,Shri Dheeraj Kumar Singh,Indian Administrative Service,Rajasthan,2020,28/06/1991,,01RJ119S03,Direct Recruitment,Male,Uttar Pradesh,...,,Serving,,,,,,,,
119,Shri Siddharth Palanichamy S,Indian Administrative Service,Rajasthan,2020,25/12/1996,,01RJ119S04,Direct Recruitment,Male,Tamil Nadu,...,,Serving,,,,,,,,
120,Shri Sohan Lal,Indian Administrative Service,Rajasthan,2020,01/07/1994,,01RJ119S05,Direct Recruitment,Male,Rajasthan,...,,Serving,,,,,,,,


In [35]:
new_df["Allotment Year"]

0       1982
1       1982
2       1982
3       1982
4       1982
        ... 
2260    1992
2261    1992
2262    1992
2263    1992
2264    1992
Name: Allotment Year, Length: 2265, dtype: object

In [26]:
new_df.to_csv("./outputs/supremo-nic-in/supremo-nic.csv", index=False)

In [49]:
info["Name"] = info["Name"][:len(info["Name"])-1]

In [18]:
df = pd.read_csv("./outputs/supremo-nic-in/supremo-nic.csv")

In [19]:
df[len(df.index)-200:]

,Name,Service,Cadre,Allotment Year,Date of Birth,Date of Joining,Identity No.,Source of Recruitment,Gender,Place of Domicile,...,Languages Known,Retirement Reason,Qualification/University/Institute,Subject,Division,Designation/Level,Ministry/Department/Office/Location,Organisation,Experience(major/minor),Period(From/To)
9492,Shri Abinash Mishra,Indian Administrative Service,Chhattisgarh,2018,02/07/1992,01/09/2018,01CG117E03,Direct Recruitment,Male,Odisha,...,NaN,Serving,B.Tech./Indian Institute of Technology Kharagp...,Mechanical engineering,First,N.A./Not Available,NaN,Cadre (AIS),/ N.Applicable/N.Available,01/09/2018 -
9493,Shri Abhishek Sharma,Indian Administrative Service,Chhattisgarh,2018,19/11/1993,01/09/2018,01CG117E04,Direct Recruitment,Male,Jammu & Kashmir,...,NaN,Serving,B.E./Jammu University|M.A./JAWAHAR LAL NEHRU U...,Civil Engg|Public Management,First with Distinction|Pass,S D M/Junior Scale|N.A./Not Available,|,Cadre (AIS)|Cadre (AIS),Land Acquision / Land Revenue Mgmt & District ...,21/09/2020 -|01/09/2018 - 30/08/2019
9494,Shri Ashutosh Dwivedi,Indian Administrative Service,Bihar,2018,23/08/1987,01/09/2018,01MN117Y01,Direct Recruitment,Male,Uttar Pradesh,...,NaN,Serving,B.Tech./Uttar Pradesh Technical University Luc...,Mechanical,First,Assistant Collector & Assistant Magistrate/Not...,Bhagalpur,Cadre(Domestic Training),Law / Public Administration,14/07/2019 - 13/08/2019
9495,Shri Aditya Vikram Yadav,Indian Administrative Service,Assam Meghalya,2018,22/07/1993,01/09/2018,01AM117C04,Direct Recruitment,Male,Delhi,...,NaN,Serving,"B.A., LL.B./National Law University Delhi/Nati...",Constitl. Law|Law,First|First,AC (UT)with DC/Not Available|N.A./Not Available,Department of Personnel/O/o Deputy Commissioner|,Cadre(Domestic Training)|Cadre (AIS),Rural Employment / Rural Dev|/ N.Applicable/N....,03/06/2019 - 25/08/2019|01/09/2018 - 25/08/2019
9496,Shri Vinod Duhan,Indian Administrative Service,Bihar,2018,11/11/1987,01/09/2018,01NL117P01,Direct Recruitment,Male,Haryana,...,NaN,Serving,"B.E./PEC,CHANDIGARH/University College",Information Technology,First,Assistant Collector & A D M/Not Available,NaN,Cadre(Domestic Training),Public Administration / Public Administration,15/07/2019 - 28/08/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9687,Ms. Gunjan Singh,Indian Administrative Service,Rajasthan,2020,29/07/1993,NaN,01RJ119S02,Direct Recruitment,Female,Uttar Pradesh,...,NaN,Serving,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9688,Shri Dheeraj Kumar Singh,Indian Administrative Service,Rajasthan,2020,28/06/1991,NaN,01RJ119S03,Direct Recruitment,Male,Uttar Pradesh,...,NaN,Serving,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9689,Shri Siddharth Palanichamy S,Indian Administrative Service,Rajasthan,2020,25/12/1996,NaN,01RJ119S04,Direct Recruitment,Male,Tamil Nadu,...,NaN,Serving,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9690,Shri Sohan Lal,Indian Administrative Service,Rajasthan,2020,01/07/1994,NaN,01RJ119S05,Direct Recruitment,Male,Rajasthan,...,NaN,Serving,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
df = df.append(new_df, ignore_index=True)

In [90]:
df.to_csv("./outputs/supremo-nic-in/supremo-nic.csv", index=False)

In [63]:
driver.close()

In [64]:
driver_profile.close()